In [1]:
import os 
import litellm
from typing import Final
from dotenv import load_dotenv
from textwrap import dedent, wrap
from pydantic import BaseModel, Field

load_dotenv(override=True)


class Dimention(BaseModel):
    meal_type: str = Field(..., description="The type of meal the user is interested in, such as 'Lite breakfast', 'Brunch'")
    dietary_restriction: str = Field(..., description="The dietary restrictions the user has, such as 'Not spicy', 'without fish or meat'")
    preparation_time: str = Field(..., description="The preparation time the user is interested in, such as '10 minutes', 'something quick', 'under 1 hour'")

    @classmethod
    def get_prompt_description(cls) -> str:
        description = []
        for field_name, field_info in cls.model_fields.items():
            description.append(f"- {field_name}: {field_info.description}")
        return "\n".join(description)


num_dimention_instances: Final[int] = 50


class DimentionList(BaseModel):
    dimention_list: list[Dimention] = Field(
        ..., 
        description=f"List of dimention instances, there should be {num_dimention_instances} instances in the list", 
        default_factory=list)


In [3]:
MODEL_NAME: Final[str] = os.environ["MODEL_NAME"]
MODEL_NAME

'gemini/gemini-2.5-flash'

In [4]:
from litellm import get_supported_openai_params

params = get_supported_openai_params(model=MODEL_NAME)

custom_llm_provider = litellm.get_llm_provider(model=MODEL_NAME)

print(custom_llm_provider)

assert "response_format" in params

print(params)

from litellm import supports_response_schema

assert supports_response_schema(model=MODEL_NAME)

('gemini-2.5-flash', 'gemini', None, None)
['temperature', 'top_p', 'max_tokens', 'max_completion_tokens', 'stream', 'tools', 'tool_choice', 'functions', 'response_format', 'n', 'stop', 'logprobs', 'frequency_penalty', 'modalities', 'parallel_tool_calls', 'web_search_options', 'reasoning_effort', 'thinking']


In [7]:
def generate_dimention_list(dimention_list: str):
    SYSTEM_PROMPT = dedent(f"""
        You are AI assistant that helps generate instances for dimentions of the queries that will be sent to the recipe chatbot. A dimention describes aspects of the query that that the users send to the chatbot. 
        Given following dimention types, generate tuple combinations of the dimentions

        Dimentions:
            meal_type: The type of meal the user is interested in, such as 'Lite breakfast', 'Brunch'
            dietary_restriction: The dietary restrictions the user has, such as 'Not spicy', 'without fish or meat', 'meaty'
            preparation_time: The preparation time the user is interested in, such as '10 minutes', 'something quick', 'under 1 hour', 'I haveall the time until retirement'

        Instructions:
            - Generate  {num_dimention_instances} tuple combinations of the dimentions.
            - Avoid duplicates. 
            - Make sure that the tuples are diverse. 

        Example output:
        - ('Lite breakfast', 'Not spicy', '10 minutes')
        - ('Snack', 'with hummus and carrots', 'instant')
    """)
    completion = litellm.completion(
        model=MODEL_NAME,
        messages=[
            {"role": "user", "content": SYSTEM_PROMPT}
        ],
    )
    return completion.choices[0].message.content

In [8]:
dimention_list = generate_dimention_list("")

print(dimention_list)


Here are 50 diverse tuple combinations of the dimensions:

1.  ('Breakfast', 'Not spicy', '10 minutes')
2.  ('Lunch', 'Vegetarian', '30 minutes')
3.  ('Dinner', 'Meaty', 'Under 1 hour')
4.  ('Snack', 'Gluten-free', 'Quick')
5.  ('Dessert', 'Dairy-free', '45 minutes')
6.  ('Brunch', 'Vegan', 'I have all the time until retirement')
7.  ('Appetizer', 'Low-carb', '15 minutes')
8.  ('Side Dish', 'High-protein', '20 minutes')
9.  ('Lite breakfast', 'Kid-friendly', 'Less than 20 minutes')
10. ('Holiday meal', 'Pescatarian', 'All day')
11. ('Party food', 'Sugar-free', 'About an hour')
12. ('Quick bite', 'Nut-free', 'Instant')
13. ('Breakfast', 'Keto', '30 minutes')
14. ('Lunch', 'Heart-healthy', 'Under 1 hour')
15. ('Dinner', 'Spicy', '45 minutes')
16. ('Snack', 'without fish or meat', '10 minutes')
17. ('Dessert', 'Meaty', 'Long prep')
18. ('Brunch', 'Paleo', 'Quick')
19. ('Appetizer', 'Low-salt', '15 minutes')
20. ('Side Dish', 'Diabetic-friendly', '20 minutes')
21. ('Lite breakfast', 'High-